<a href="https://colab.research.google.com/github/pri-nitta/deteccao_anomalias/blob/main/deteccao_anomalias_em_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from google.colab import files
from base64 import b64encode
from IPython.display import HTML

Upload do vídeo no Colab

In [ ]:
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

Abrindo e configurando o vídeo de saída

In [ ]:
cap = cv2.VideoCapture(video_path)


fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# manter a mesma fluidez, retorna o mesmo fps
fps = int(cap.get(cv2.CAP_PROP_FPS))


width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('output_anomaly_detection.mp4', fourcc, fps, (width, height)

Subtração de fundo para detectar objetos estáticos

In [ ]:
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Aplica subtração de fundo
    fgmask = fgbg.apply(frame)

    # Remove ruídos pequenos
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    fgmask_cleaned = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)

    # Encontra contornos nos objetos detectados
    contours, _ = cv2.findContours(fgmask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        if cv2.contourArea(contour) > 500:  # Define um limite mínimo para considerar como objeto
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)  # Destaca o objeto com retângulo vermelho
            cv2.putText(frame, "ANOMALY DETECTED", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Salva o frame processado
    out.write(frame)

cap.release()
out.release()

Exibindo o vídeo processado

In [ ]:
mp4 = open('output_anomaly_detection.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width="640" height="480" controls><source src="{data_url}" type="video/mp4"></video>')